In [91]:
#%matplotlib notebook

In [1]:
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import numpy as np
# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [3]:
from netCDF4 import Dataset

In [4]:
import glob

In [5]:
import re

In [6]:
import datetime

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet

In [8]:
def hvel(r):
    x=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    return (bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**x

In [9]:
def pres(r):
    return pch+dph*np.exp(-(rmaxh/r)**bh)

In [10]:
def hvel2d(l1,l2,vtx,vty):
    r=np.sqrt(l1**2+l2**2)
    xh=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    ur=(bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**xh
    theta=np.arctan2(l2,l1)
    ux=-ur*np.sin(theta)
    uy=ur*np.cos(theta)
    return ux+vtx,uy+vty, pres(r)

In [11]:
path='../BestTrack/'

In [12]:
data=pd.read_csv(path+'Allstorms.ibtracs_all.v03r09.csv', header=1, low_memory=False)

In [13]:
data.head()

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,...,jtwc_??_wrad34_rad3,jtwc_??_wrad34_rad4,jtwc_??_wrad50_rad1,jtwc_??_wrad50_rad2,jtwc_??_wrad50_rad3,jtwc_??_wrad50_rad4,jtwc_??_wrad64_rad1,jtwc_??_wrad64_rad2,jtwc_??_wrad64_rad3,jtwc_??_wrad64_rad4
0,NaN,Year,#,BB,BB,NaN,YYYY-MM-DD HH:MM:SS,NaN,deg_north,deg_east,...,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile
1,1842298N11080,1842,01,NI,BB,NOT NAMED,1842-10-25 06:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
2,1842298N11080,1842,01,NI,BB,NOT NAMED,1842-10-25 12:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
3,1842298N11080,1842,01,NI,AS,NOT NAMED,1842-10-25 18:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
4,1842298N11080,1842,01,NI,AS,NOT NAMED,1842-10-26 00:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000


In [14]:
storms=Dataset(path+'Allstorms.ibtracs_all.v03r09.nc')

In [15]:
storms.variables.keys()

[u'storm_sn',
 u'name',
 u'numObs',
 u'season',
 u'track_type',
 u'genesis_basin',
 u'num_basins',
 u'basin',
 u'wind_avg_period',
 u'source',
 u'source_time',
 u'lat_for_mapping',
 u'lon_for_mapping',
 u'nature_for_mapping',
 u'source_lat',
 u'source_lon',
 u'source_wind',
 u'source_pres',
 u'dist2land',
 u'landfall']

# ANALYSIS STARTS HERE

In [16]:
hurName='PAM'
year = '2015'

In [17]:
hurdata = data[data['Name'].str.contains(hurName,na=False) & data['ISO_time'].str.contains(year,na=False)]

In [18]:
hurdata.iloc[0]

Serial_Num                     2015066S08170
Season                                  2015
Num                                       05
Basin                                     SP
Sub_basin                                 MM
Name                                     PAM
ISO_time                 2015-03-07 06:00:00
Nature                                    NR
Latitude                               -8.00
Longitude                             169.50
Wind(WMO)                               20.0
Pres(WMO)                             1000.0
Center                                  nadi
Wind(WMO) Percentile                   4.063
Pres(WMO) Percentile                  10.347
Track_type                              main
Latitude_for_mapping                   -8.00
Longitude_for_mapping                 169.50
Current Basin                             SP
hurdat_atl_lat                        -999.0
hurdat_atl_lon                        -999.0
hurdat_atl_grade                      -999.0
hurdat_atl

In [19]:
hurdata.shape

(79, 200)

In [20]:
ntc=hurdata['Basin'].unique()

In [21]:
ntc

array([' SP'], dtype=object)

In [22]:
hur1=hurdata.loc[hurdata['Basin']==ntc[0]]

In [23]:
hur1=hur1.dropna()

In [24]:
hur1[['Longitude','Latitude']]=hur1[['Longitude','Latitude']].apply(pd.to_numeric)

In [28]:
hur1.Longitude.loc[hur1.Longitude<0.]=hur1.Longitude.loc[hur1.Longitude<0.].values+360.

In [29]:
plt.plot(hur1['Longitude'],hur1['Latitude'],'o--')
mplleaflet.display()

In [30]:
idf=hur1.iloc[0].Serial_Num

In [31]:
filename=glob.glob('../BestTrack/ibtracs/{}*'.format(idf))

In [32]:
dat=Dataset(filename[0])

In [33]:
dat.variables.keys()

[u'nobs_wmo',
 u'nobs',
 u'genesis_basin',
 u'num_basins',
 u'season',
 u'source',
 u'track_from_source',
 u'name_from_source',
 u'time_wmo',
 u'isotime_wmo',
 u'source_wmo',
 u'averaging_time',
 u'lat_wmo',
 u'lon_wmo',
 u'wind_wmo',
 u'pres_wmo',
 u'basin_wmo',
 u'sub_basin_wmo',
 u'nature_wmo',
 u'dist2land_wmo',
 u'landfall_wmo',
 u'pres_perc_wmo',
 u'wind_perc_wmo',
 u'time',
 u'isotime',
 u'lat_for_mapping',
 u'lon_for_mapping',
 u'wind_for_mapping',
 u'pres_for_mapping',
 u'nature_for_mapping',
 u'dist2land',
 u'landfall',
 u'lat_from_source',
 u'lon_from_source',
 u'wind_from_source',
 u'wind_avg_period',
 u'wind_perc_from_source',
 u'pres_from_source',
 u'pres_perc_from_source',
 u'basin',
 u'sub_basin',
 u'pos_stdev',
 u'pos_num',
 u'pos_flag',
 u'wind_quality',
 u'wind_stdev',
 u'wind_condition',
 u'wind_num',
 u'wind_flag',
 u'wind_maximum',
 u'wind_minimum',
 u'wind_median',
 u'pres_quality',
 u'pres_stdev',
 u'pres_condition',
 u'pres_flag',
 u'pres_num',
 u'pres_maximum'

In [34]:
wradii=dat['jtwc_sh_wrad'][:13]

In [35]:
wradiinp=np.array(wradii)

In [36]:
w34= wradiinp[:,0,:]

In [37]:
w34ne=w34[:,0]
w34nw=w34[:,1]
w34se=w34[:,2]
w34sw=w34[:,3]

In [38]:
w50=wradiinp[:,1,:]

In [39]:
w50ne=w50[:,0]
w50nw=w50[:,1]
w50se=w50[:,2]
w50sw=w50[:,3]

In [40]:
w64=wradiinp[:,2,:]

In [41]:
w64ne=w64[:,0]
w64nw=w64[:,1]
w64se=w64[:,2]
w64sw=w64[:,3]

In [42]:
isot=dat['isotime'][:13]

In [43]:
isot1=[''.join(x) for x in isot]

In [44]:
time=[re.sub(r'[-: ]+','',x)[:-4] for x in isot1]

In [45]:
lat=dat['jtwc_sh_lat'][:13]
lon=dat['jtwc_sh_lon'][:13]
pcenter=dat['jtwc_sh_pres'][:13]
vmaxh=dat['jtwc_sh_wind'][:13] # 10 minute wind in Knots
rmaxh=dat['jtwc_sh_rmw'][:13]*nm2m

In [89]:
dic={'time':time, 'lat':lat,'lon':lon,'pcenter':pcenter,'vmax':vmaxh, 'rmax':rmaxh, '34ne': w34ne, '34nw': w34nw, '34se':w34se, '34sw':w34sw, '50ne': w50ne, '50nw': w50nw, '50se':w50se, '50sw':w50sw,'64ne': w64ne, '64nw': w64nw, '64se':w64se, '64sw':w64sw} 

In [129]:
inpData=pd.DataFrame(dic)

In [130]:
inpData=inpData.apply(pd.to_numeric)

In [131]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,time,vmax
0,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,9.969210e+36,9.969210e+36,-32767,-60684484.0,2015030706,-32767
1,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,9.969210e+36,9.969210e+36,-32767,-60684484.0,2015030712,-32767
2,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,9.969210e+36,9.969210e+36,-32767,-60684484.0,2015030718,-32767
3,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,9.969210e+36,9.969210e+36,-32767,-60684484.0,2015030800,-32767
4,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,-32767,9.969210e+36,9.969210e+36,-32767,-60684484.0,2015030806,-32767
5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-7.500000e+00,1.689000e+02,1004,92600.0,2015030812,25
6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-7.800000e+00,1.692000e+02,1000,92600.0,2015030818,30
7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-8.100000e+00,1.696000e+02,1000,92600.0,2015030900,30
8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-8.500000e+00,1.701000e+02,996,92600.0,2015030906,35
9,30,30,30,25,-1,-1,-1,-1,-1,-1,-1,-1,-9.000000e+00,1.706000e+02,989,55560.0,2015030912,45


In [132]:
inpData.loc[inpData['lat'] == 9.969210e+36, 'lat'] = np.nan
inpData.loc[inpData['lon'] == 9.969210e+36, 'lon'] = np.nan
inpData.loc[inpData['pcenter'] == -32767., 'pcenter'] = np.nan
inpData.loc[inpData['rmax'] == -60684484., 'rmax'] = np.nan
inpData.loc[inpData['vmax'] == -32767, 'vmax'] = np.nan
inpData.loc[inpData['34ne'] == -32767, '34ne']=np.nan
inpData.loc[inpData['34nw'] == -32767, '34nw']=np.nan
inpData.loc[inpData['34se'] == -32767, '34se']=np.nan
inpData.loc[inpData['34sw'] == -32767, '34sw']=np.nan
inpData.loc[inpData['50ne'] == -32767, '50ne']=np.nan
inpData.loc[inpData['50nw'] == -32767, '50nw']=np.nan
inpData.loc[inpData['50se'] == -32767, '50se']=np.nan
inpData.loc[inpData['50sw'] == -32767, '50sw']=np.nan
inpData.loc[inpData['64ne'] == -32767, '64ne']=np.nan
inpData.loc[inpData['64nw'] == -32767, '64nw']=np.nan
inpData.loc[inpData['64se'] == -32767, '64se']=np.nan
inpData.loc[inpData['64sw'] == -32767, '64sw']=np.nan

In [133]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,time,vmax
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015030706,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015030712,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015030718,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015030800,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015030806,NaN
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-7.5,168.899994,1004.0,92600.0,2015030812,25.0
6,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-7.8,169.199997,1000.0,92600.0,2015030818,30.0
7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-8.1,169.600006,1000.0,92600.0,2015030900,30.0
8,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-8.5,170.100006,996.0,92600.0,2015030906,35.0
9,30.0,30.0,30.0,25.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-9.0,170.600006,989.0,55560.0,2015030912,45.0


In [134]:
inpData.loc[inpData['34ne'] == -1, '34ne']=0
inpData.loc[inpData['34nw'] == -1, '34nw']=0
inpData.loc[inpData['34se'] == -1, '34se']=0
inpData.loc[inpData['34sw'] == -1, '34sw']=0
inpData.loc[inpData['50ne'] == -1, '50ne']=0
inpData.loc[inpData['50nw'] == -1, '50nw']=0
inpData.loc[inpData['50se'] == -1, '50se']=0
inpData.loc[inpData['50sw'] == -1, '50sw']=0
inpData.loc[inpData['64ne'] == -1, '64ne']=0
inpData.loc[inpData['64nw'] == -1, '64nw']=0
inpData.loc[inpData['64se'] == -1, '64se']=0
inpData.loc[inpData['64sw'] == -1, '64sw']=0

In [135]:
inpData=inpData.dropna()

In [136]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,time,vmax
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.5,168.899994,1004.0,92600.0,2015030812,25.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.8,169.199997,1000.0,92600.0,2015030818,30.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.1,169.600006,1000.0,92600.0,2015030900,30.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.5,170.100006,996.0,92600.0,2015030906,35.0
9,30.0,30.0,30.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,170.600006,989.0,55560.0,2015030912,45.0
10,70.0,60.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,-9.3,170.500000,982.0,37040.0,2015030918,55.0
11,70.0,60.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,-9.7,170.500000,978.0,37040.0,2015031000,60.0
12,70.0,65.0,65.0,75.0,25.0,20.0,20.0,25.0,0.0,0.0,0.0,0.0,-10.2,170.500000,974.0,37040.0,2015031006,65.0


In [137]:
dph=pn-inpData.pcenter*100.

In [138]:
dph

5      600.0
6     1000.0
7     1000.0
8     1400.0
9     2100.0
10    2800.0
11    3200.0
12    3600.0
Name: pcenter, dtype: float64

In [139]:
inpData=inpData.assign(dp=dph)

In [140]:
inpData.iloc[7]

34ne       7.000000e+01
34nw       6.500000e+01
34se       6.500000e+01
34sw       7.500000e+01
50ne       2.500000e+01
50nw       2.000000e+01
50se       2.000000e+01
50sw       2.500000e+01
64ne       0.000000e+00
64nw       0.000000e+00
64se       0.000000e+00
64sw       0.000000e+00
lat       -1.020000e+01
lon        1.705000e+02
pcenter    9.740000e+02
rmax       3.704000e+04
time       2.015031e+09
vmax       6.500000e+01
dp         3.600000e+03
Name: 12, dtype: float64

### create inpData file

In [141]:
inpData['time']=pd.to_datetime(inpData['time'],format='%Y%m%d%H')

In [142]:
inpData.head()

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,time,vmax,dp
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.5,168.899994,1004.0,92600.0,2015-03-08 12:00:00,25.0,600.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.8,169.199997,1000.0,92600.0,2015-03-08 18:00:00,30.0,1000.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.1,169.600006,1000.0,92600.0,2015-03-09 00:00:00,30.0,1000.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.5,170.100006,996.0,92600.0,2015-03-09 06:00:00,35.0,1400.0
9,30.0,30.0,30.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,170.600006,989.0,55560.0,2015-03-09 12:00:00,45.0,2100.0


In [143]:
#inpData['time']=inpData['t']-inpData['t'][0]

In [144]:
#inpData['time']=inpData['time'] / pd.Timedelta('1 hour')

In [145]:
#inpData=inpData.set_index('time')
#inpData

## convert to SI

In [146]:
inpData['vmax']=inpData['vmax']*kt2ms

In [147]:
inpData[['64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']]=inpData[['64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']]*nm2m

In [148]:
inpData[3:4]

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,time,vmax,dp
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.5,170.100006,996.0,92600.0,2015-03-09 06:00:00,18.005556,1400.0


## save inpData file

In [149]:
column_order=['time','lat','long','dp','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [150]:
inpData.to_csv('tmp/inpData.txt',index=False, columns=column_order, sep='\t')

### create bulInfo.txt file

In [151]:
tt=pd.to_datetime(isot1[0])

In [152]:
tt=datetime.datetime.strftime(tt,'%d %b %Y %H:%M:%S')

In [153]:
dic0={'advNo':[1],'tShift':[0],'$date':tt,'land':[1],'notes':[0]}

In [154]:
bul=pd.DataFrame.from_dict(dic0)

In [155]:
bul

,$date,advNo,land,notes,tShift
0,07 Mar 2015 06:00:00,1,1,0,0


In [185]:
bul.to_csv('tmp/bulInfo.txt',index=False, columns=['advNo','tShift','$date','land','notes'], sep='\t')

### create info.pr file

## compute b,k

In [47]:
plt.figure(figsize=(10,8))
plt.plot(rh/1000,hvel(rh))
plt.xlabel('km',fontsize=18)
plt.ylabel('Velocity [m/s]',fontsize=18)
plt.title('Velocity vs radius')

NameError: global name 'kh' is not defined

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(rh/1000,pres(rh)/100)

plt.xlabel('km',fontsize=18)
plt.ylabel('Pressure [PA]',fontsize=18)
plt.title('Pressure vs radius')


plt.show()

In [ ]:
ni,nj=1000,1000
xl=np.linspace(-maxR, maxR,ni+1)
yl=np.linspace(-maxR, maxR,nj+1)
q1,q2=np.meshgrid(xl,yl)

In [ ]:
zx,zy, pr=hvel2d(q1,q2,vtx,vty)

In [ ]:
plt.figure(figsize=(10,8))
plt.contourf(q1/1000,q2/1000,pr/100,20)
plt.xlabel('km',fontsize=18)
plt.ylabel('Km',fontsize=18)
plt.title('Pressure contours')


plt.colorbar(label='PA')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(q1[0,:]/1000,pr[500,:]/100)

plt.xlabel('km',fontsize=18)
plt.ylabel('Pressure',fontsize=18)
plt.title('Pressure vs radius')


plt.show()

In [ ]:

vel=np.sqrt(zx**2+zy**2)

In [ ]:
plt.figure(figsize=(10,8))
plt.contourf(q1/1000,q2/1000,vel,20)
plt.streamplot(q1/1000,q2/1000,zx,zy, color=vel, linewidth=0.1*vel)
plt.xlabel('km',fontsize=18)
plt.ylabel('Km',fontsize=18)
plt.title('Velocity vs radius')

plt.colorbar(label='m/s')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(q1[0,:]/1000,vel[ni/2,:],label='horizontal')
plt.plot(q2[:,0]/1000,vel[:,nj/2],label='verical')

plt.xlabel('km',fontsize=18)
plt.ylabel('Velocity [m/s]',fontsize=18)
plt.title('Velocity vs radius')


plt.legend(loc=0)
plt.show()

In [ ]:
np.savetxt('tmp/phol',[q1[0,:],pr[500,:]])
np.savetxt('tmp/uhol',[q1[0,:],vel[ni/2,:],vel[:,nj/2]])

In [ ]:
pd.DataFrame({'Min Pressure':[pch/100],'Max Velocity':[vmaxh]})